In [1]:
%matplotlib inline
from collections import OrderedDict
import sqlite3
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from lifelines import KaplanMeierFitter, CoxPHFitter
from lifelines.utils import concordance_index, k_fold_cross_validation
import patsy as pt
from sklearn.model_selection import train_test_split

# suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Set default styles for plotting via pandas, seaborn and matplotlib
pd.set_option('display.mpl_style', 'default')
pd.set_option('display.notebook_repr_html', True)
sns.set(style='darkgrid')
cmap_clrbld = ['#777777','#E69F00','#56B4E9','#D3C511'
               ,'#009E73','#8D42F0','#0072B2','#D55E00','#CC79A7']
plt.rcParams['axes.color_cycle'] = cmap_clrbld
plt.rcParams['figure.figsize'] = 10, 6

np.random.seed(0)

In [2]:
df = pd.read_csv('../data/15_16_17/drive_survival_prepared.csv'
                 ,index_col='diskid', parse_dates=['mindate','maxdate'])
df.head()

,model,mindate,maxdate,nrecords,minhours,maxhours,failed,manufacturer,capacity,mindateym,maxdateym
diskid,,,,,,,,,,,
13H2B97AS,DT01ACA300,2015-04-21,2017-07-25,911,10153,29961,0,TOSHIBA,3.0TB,201504,201707
13H32WEAS,DT01ACA300,2015-01-01,2017-08-03,1030,14765,37431,0,TOSHIBA,3.0TB,201501,201708
13H6A21GS,DT01ACA300,2015-01-01,2017-08-18,1045,14133,37155,0,TOSHIBA,3.0TB,201501,201708
13H7X2HAS,DT01ACA300,2015-12-17,2017-04-26,583,9775,21667,0,TOSHIBA,3.0TB,201512,201704
13H80PNGS,DT01ACA300,2015-01-01,2015-08-13,225,11765,17138,0,TOSHIBA,3.0TB,201501,201508


In [3]:
cox_data = df[['model', 'manufacturer', 'nrecords', 'minhours', 'maxhours', 'failed']]
cox_data = pd.get_dummies(cox_data, columns=['model', 'manufacturer'])
print(cox_data.shape)
cox_data.head()

(117840, 66)


,nrecords,minhours,maxhours,failed,model_DT01ACA300,model_HD154UI,model_HDS5C3030ALA630,model_HDS5C3030BLE630,model_HDS5C4040ALE630,model_HDS722020ALA330,...,model_WD5000LPCX,model_WD5000LPVX,model_WD5002ABYS,model_WD5003ABYX,model_WD60EFRX,manufacturer_HGST,manufacturer_SAMSUNG,manufacturer_SEAGATE,manufacturer_TOSHIBA,manufacturer_WDC
diskid,,,,,,,,,,,,,,,,,,,,,
13H2B97AS,911,10153,29961,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
13H32WEAS,1030,14765,37431,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
13H6A21GS,1045,14133,37155,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
13H7X2HAS,583,9775,21667,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
13H80PNGS,225,11765,17138,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [4]:
cox_tr, cox_va = train_test_split(cox_data, test_size=0.2, random_state=15125)

In [5]:
cox = CoxPHFitter()
cox.fit(cox_tr, duration_col='maxhours', event_col='failed', show_progress=True, step_size=0.01)

Iteration 1: norm_delta = 67450.71367, step_size = 0.01000, ll = -32263.29733, seconds_since_start = 2.4
Iteration 2: norm_delta = 2098.36963, step_size = 0.00250, ll = -31999.56430, seconds_since_start = 4.8
Iteration 3: norm_delta = 836.33448, step_size = 0.00075, ll = -31936.81294, seconds_since_start = 8.3
Iteration 4: norm_delta = 0.00259, step_size = 0.00027, ll = -31918.21023, seconds_since_start = 11.1
Iteration 5: norm_delta = 274.78772, step_size = 0.00039, ll = -31911.53705, seconds_since_start = 14.4
Iteration 6: norm_delta = 0.00486, step_size = 0.00011, ll = -31901.94133, seconds_since_start = 16.9
Iteration 7: norm_delta = 0.00164, step_size = 0.00013, ll = -31899.12534, seconds_since_start = 19.6
Iteration 8: norm_delta = 17431.19996, step_size = 0.00019, ll = -31895.81567, seconds_since_start = 22.0
Iteration 9: norm_delta = 0.03827, step_size = 0.00006, ll = -31891.05311, seconds_since_start = 25.3
Iteration 10: norm_delta = 0.00166, step_size = 0.00007, ll = -31889.6

<lifelines.CoxPHFitter: fitted with 94272 observations, 91238 censored>

In [6]:
cox.print_summary()

n=94272, number of events=3034

                                coef  exp(coef)           se(coef)       z      p           lower 0.95          upper 0.95   
nrecords                     -0.0003     0.9997             0.0001 -2.3075 0.0210              -0.0005             -0.0000  *
minhours                      0.0000     1.0000             0.0000  0.5788 0.5627              -0.0000              0.0000   
model_DT01ACA300              0.0055     1.0055        434350.2752  0.0000 1.0000         -851310.8905         851310.9015   
model_HD154UI         -12869273.4444     0.0000 8587131123476.7383 -0.0000 1.0000 -16830480601810.8242 16830454863263.9336   
model_HDS5C3030ALA630         0.2482     1.2817                nan     nan    nan                  nan                 nan   
model_HDS5C3030BLE630         0.2624     1.3000                nan     nan    nan                  nan                 nan   
model_HDS5C4040ALE630         0.3273     1.3872                nan     nan    nan     

Likelihood ratio test = 764.716 on 64 df, p=0.00000


In [7]:
hazard_tr, hazard_va = cox.predict_partial_hazard(cox_tr), cox.predict_partial_hazard(cox_va)
score_tr = concordance_index(cox_tr['maxhours'], -hazard_tr, cox_tr['failed'])
score_va = concordance_index(cox_va['maxhours'], -hazard_va, cox_va['failed'])
print(score_tr, score_va)

0.958537495603 0.950160532711
